In [0]:
OUTPUTPath = './output/'
!mkdir ./output

In [2]:
# Save model to your Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


#Prepare Pretrained Bert model

In [3]:
!pip install -q keras-bert
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -o uncased_L-12_H-768_A-12.zip -d './gdrive/My Drive/NLP/Final-Project/data/'

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: ./gdrive/My Drive/NLP/Final-Project/data/uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: ./gdrive/My Drive/NLP/Final-Project/data/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: ./gdrive/My Drive/NLP/Final-Project/data/uncased_L-12_H-768_A-12/vocab.txt  
  inflating: ./gdrive/My Drive/NLP/Final-Project/data/uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: ./gdrive/My Drive/NLP/Final-Project/data/uncased_L-12_H-768_A-12/bert_config.json  


In [4]:
from keras_bert import load_vocabulary, load_trained_model_from_checkpoint, Tokenizer, get_checkpoint_paths
bert_path = './gdrive/My Drive/NLP/Final-Project/data/uncased_L-12_H-768_A-12'
paths = get_checkpoint_paths(bert_path)
model = load_trained_model_from_checkpoint(paths.config, paths.checkpoint, seq_len=83)
model.summary(line_length=120)

Using TensorFlow backend.
W0622 14:51:20.410980 140087159367552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0622 14:51:20.453547 140087159367552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0622 14:51:20.504328 140087159367552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0622 14:51:20.505295 140087159367552 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default inste

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
Input-Token (InputLayer)               (None, 83)                 0                                                     
________________________________________________________________________________________________________________________
Input-Segment (InputLayer)             (None, 83)                 0                                                     
________________________________________________________________________________________________________________________
Embedding-Token (TokenEmbedding)       [(None, 83, 768), (30522,  23440896      Input-Token[0][0]                       
________________________________________________________________________________________________________________________
Embedding-Segment (Embedding)   

In [0]:
token_dict = load_vocabulary(paths.vocab)


#Load clean text data

In [20]:
import re
import numpy as np
import os
import pandas as pd
DATA_SET_DIR = './gdrive/My Drive/NLP/Final-Project/data/'
TRAINING_DATA_PATH = DATA_SET_DIR+'olid-training-clean.csv'
TEST_PATH_DICT = {}
TEST_A_DATA_PATH = DATA_SET_DIR+'testset-levela-clean.tsv'
TEST_B_DATA_PATH = DATA_SET_DIR+'testset-levelb-clean.tsv'
TEST_C_DATA_PATH = DATA_SET_DIR+'testset-levelc-clean.tsv'
TEST_PATH_DICT['subtask_a'] = TEST_A_DATA_PATH
TEST_PATH_DICT['subtask_b'] = TEST_B_DATA_PATH
TEST_PATH_DICT['subtask_c'] = TEST_C_DATA_PATH

data={}
data['train'] = {}
data['validation'] = {}
data['test'] = {}
task_list = ['subtask_a','subtask_b','subtask_c']
train = pd.read_csv(TRAINING_DATA_PATH,sep='\t', index_col=False)
for subtask in task_list:
    data['test'][subtask] = pd.read_csv(TEST_PATH_DICT[subtask],sep='\t', index_col=False)
train.head(3)

,id,tweet_exp,subtask_a,subtask_b,subtask_c
0,86426,@USER She should ask a few native Americans wh...,OFF,UNT,NaN
1,90194,@USER @USER Go home you are drunk!!! @USER #MA...,OFF,TIN,IND
2,16820,Amazon is investigating Chinese employees who ...,NOT,NaN,NaN


#Text Data Preprocessing

In [0]:
def clean_tweet(text):
#     def handle_emoji(text):
#       # Dictionnary of "important" emojis : 
#       emoji_dict =  {'♥️': ' love ',
#                      '❤️' : ' love ',
#                      '❤' : ' love ',
#                      '😘' : ' kisses ',
#                     '😭' : ' cry ',
#                     '💪' : ' strong ',
#                     '🌍' : ' earth ',
#                     '💰' : ' money ',
#                     '👍' : ' ok ',
#                      '👌' : ' ok ',
#                     '😡' : ' angry ',
#                     '🍆' : ' dick ',
#                     '🤣' : ' haha ',
#                     '😂' : ' haha ',
#                     '🖕' : ' fuck you '}

#       for cha in emoji_dict:
#           text = re.compile(str(cha)).sub(str(emoji_dict[cha]),text)
#       # Remove ALL emojis
#       text = emoji.get_emoji_regexp().sub(r' ',text) 
#       text = re.compile("([\U0001f3fb-\U0001f3ff])").sub(r'',text) 
#       text = re.compile("([\U00010000-\U0010ffff])").sub(r'',text) 
#       text = re.compile("(\u00a9|\u00ae|[\u2000-\u3300]|\ud83c[\ud000-\udfff]|\ud83d[\ud000-\udfff]|\ud83e[\ud000-\udfff])").sub(r'',text)

#       # Add Space between  the Emoji Expressions : 
#       text = re.compile("([\U00010000-\U0010ffff])").sub(r' \1 ',text) 
#       return text
    ''' Function that is applied to every to tweet in the dataset '''

    # =========== TEXT ===========
    # Replace @USER by <user>
    text = re.compile(r'@USER').sub(r'<user>',text)

    # Replace URL by <url>
    text = re.compile(r'URL').sub(r'<url>',text)

    # Remove numbers :
    text = re.compile(r'[0-9]+').sub(r' ',text)

    # Remove some special characters
    text = re.compile(r'([\xa0_\{\}\[\]¬•$,:;/@#|\^*%().~`”"“-])').sub(r' ',text) 

    # Space the special characters with white spaces
    text = re.compile(r'([$&+,:;=?@#|.^*()%!"’“-])').sub(r' \1 ',text)

    # Replace some special characters : 
    replace_dict = {r'&' : 'and' , 
                    r'\+' : 'plus'}
    for cha in replace_dict:
        text = re.compile(str(cha)).sub(str(replace_dict[cha]),text)

    # Handle Emoji : translate some and delete the others
#     text = handle_emoji(text)

    # Word delengthening : 
    text = re.compile(r'(.)\1{3,}').sub(r'\1\1',text)

    # Cut the words with caps in them : 
    text = re.compile(r'([a-z]+|[A-Z]+|[A-Z][a-z]+)([A-Z][a-z]+)').sub(r'\1 \2',text)
    text = re.compile(r'([a-z]+|[A-Z]+|[A-Z][a-z]+)([A-Z][a-z]+)').sub(r'\1 \2',text)        
    # =========== TOKENS ===========
    # TOKENIZE 
    text = text.split(' ')

    # Remove white spaces tokens
    text = [text[i] for i in range(len(text)) if text[i] != ' ']

    # Remove empty tokens
    text = [text[i] for i in range(len(text)) if text[i] != '']

    # Remove repetition in tokens (!!! => !)
    text = [text[i] for i in range(len(text)) if text[i] != text[i-1]]

    #  Handle the ALL CAPS Tweets 
    ### if ratio of caps in the word > 75% add allcaps tag <allcaps>
    caps_r = np.mean([text[i].isupper() for i in range(len(text))])
    if caps_r > 0.6 : 
        text.append('<allcaps>')

    # Lower Case : 
    text = [text[i].lower() for i in range(len(text))]

    return ' '.join(text)

In [7]:
import re, string
def _run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    def _is_punctuation(char):
        # print('[%s]+' % re.escape(string.punctuation))
        if re.match(r'[\!\"\$\%\&\(\)\*\+\,\-\.\/\:\;\<\=\>\?\[\\\]\^_\`\{\|\}\~]+',char):
            return True
        else:
            return False
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char):
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1
    text_list = ["".join(x) for x in output]
    text = ' '.join(text_list)
    text = text.split(' ')
    return ' '.join(text)
print(_run_split_on_punc("@USER @USER @USER @USER LOL!!!   Throwing the BULLSHIT Flag on such nonsense!!  #PutUpOrShutUp"))

@USER @USER @USER @USER LOL ! ! !    Throwing the BULLSHIT Flag on such nonsense ! !   #PutUpOrShutUp


In [0]:
train.tweet_exp = train.tweet_exp.apply(_run_split_on_punc)
train.tweet_exp = train.tweet_exp.apply(clean_tweet)
for subtask in task_list:
    data['test'][subtask].tweet_exp = data['test'][subtask].tweet_exp.apply(_run_split_on_punc)
    data['test'][subtask].tweet_exp = data['test'][subtask].tweet_exp.apply(clean_tweet)

#Prepare Bert Embeddings

In [0]:
# bert
token_dict = load_vocabulary(paths.vocab)
xtrain_list = train.tweet_exp.tolist()
tokenizer = Tokenizer(token_dict)
indice=[]
segments = []
for line in xtrain_list:
#   tokens = tokenizer.tokenize(line)
  idx, seg = tokenizer.encode(first=line, max_len=83)
  indice.append(idx)
  segments.append(seg)
predicts1 = model.predict([np.array(indice[:5000]), np.array(segments[:5000])])
predicts2 = model.predict([np.array(indice[5000:]), np.array(segments[5000:])])

In [12]:
predicts = np.concatenate((predicts1,predicts2),axis=0)
predicts.shape

(13240, 83, 768)

In [0]:
np.save('./gdrive/My Drive/NLP/Final-Project/data/train_bert_vector',predicts)

In [0]:
del predicts

In [15]:
for subtask in task_list:
    data['test'][subtask+'_list'] = data['test'][subtask].tweet_exp.tolist()
    indice=[]
    segments = []
    for line in data['test'][subtask+'_list']:
    #   tokens = tokenizer.tokenize(line)
      idx, seg = tokenizer.encode(first=line, max_len=83)
      indice.append(idx)
      segments.append(seg)
    predicts = model.predict([np.array(indice), np.array(segments)])
    print(predicts.shape)
    np.save('./gdrive/My Drive/NLP/Final-Project/data/test_bert_vector_'+subtask,predicts)
    del predicts

(860, 83, 768)
(240, 83, 768)
(213, 83, 768)


In [0]:
del data

#Load Bert Embeddings

In [0]:
import os
import pandas as pd
import numpy as np
DATA_SET_DIR = './gdrive/My Drive/NLP/Final-Project/data/'
TRAINING_DATA_PATH = DATA_SET_DIR+'train_bert_vector.npy'
TEST_PATH_DICT = {}
TEST_A_DATA_PATH = DATA_SET_DIR+'test_bert_vector_subtask_a.npy'
TEST_B_DATA_PATH = DATA_SET_DIR+'test_bert_vector_subtask_b.npy'
TEST_C_DATA_PATH = DATA_SET_DIR+'test_bert_vector_subtask_c.npy'
TEST_PATH_DICT['subtask_a'] = TEST_A_DATA_PATH
TEST_PATH_DICT['subtask_b'] = TEST_B_DATA_PATH
TEST_PATH_DICT['subtask_c'] = TEST_C_DATA_PATH

bert_data={}
bert_data['train'] = {}
bert_data['validation'] = {}
bert_data['test'] = {}
bert_data['train'] = np.load(TRAINING_DATA_PATH)
for subtask in task_list:
    bert_data['test'][subtask] = np.load(TEST_PATH_DICT[subtask])

In [21]:
classes_dict = {}
classes_dict['subtask_a'] = {'NOT' : 0 ,'OFF' : 1}
classes_dict['subtask_b'] = {'UNT' : 0 ,'TIN' : 1}
classes_dict['subtask_c'] = {'IND' : 0 ,'OTH' : 1, 'GRP' : 2}
for subtask in task_list:
  data['train'][subtask]=pd.DataFrame()
  idx = train[train[subtask].notnull()]
  data['train'][subtask+'_idx'] = idx.index
  data['train'][subtask]= idx[subtask]
#     data['train'][subtask]=[]
#     for label in classes_dict[subtask]:

#         data['train'][subtask].append(data_label.index)
#     data['train'][subtask] = 
#     print(subtask,":")
  print(data['train'][subtask].value_counts())
#     print(len(data['train'][subtask]))
#     print('-'*10)
data['train']['subtask_b'].head(5)

NOT    8840
OFF    4400
Name: subtask_a, dtype: int64
TIN    3876
UNT     524
Name: subtask_b, dtype: int64
IND    2407
GRP    1074
OTH     395
Name: subtask_c, dtype: int64


0    UNT
1    TIN
3    UNT
5    TIN
6    UNT
Name: subtask_b, dtype: object

In [22]:
# 將分類標籤對應到剛定義的數字
for subtask in task_list:
  data['train'][subtask+'_y'] = data['train'][subtask].apply(lambda x: classes_dict[subtask][x])
  print(data['train'][subtask+'_y'].value_counts())
  data['train'][subtask+'_y'] = np.asarray(data['train'][subtask+'_y']).astype('float32')
  print(data['train'][subtask+'_y'][:5])

0    8840
1    4400
Name: subtask_a, dtype: int64
[1. 1. 0. 1. 0.]
1    3876
0     524
Name: subtask_b, dtype: int64
[0. 1. 0. 1. 0.]
0    2407
2    1074
1     395
Name: subtask_c, dtype: int64
[0. 1. 2. 0. 0.]


#Prepare Validation Data

In [23]:
from sklearn.model_selection import train_test_split
NUM_LSTM_UNITS = 128
# y_train = keras.utils.to_categorical(y_train)

VALIDATION_RATIO = 0.15

RANDOM_STATE = 9527
for subtask in task_list:
#   if subtask == 'subtask_c':
#     data['train'][subtask+'_y'] = keras.utils.to_categorical(data['train'][subtask+'_y'])
  
  data['train'][subtask+'_x_train'], data['validation'][subtask+'_x_val'], \
  data['train'][subtask+'_y_train'], data['validation'][subtask+'_y_val'] =train_test_split(data['train'][subtask+'_idx'],data['train'][subtask+'_y'],test_size=VALIDATION_RATIO,random_state=RANDOM_STATE)
  print("Training Set",'----',subtask)
  print("-" * 10)
  print("x_train: ", data['train'][subtask+'_x_train'].shape)
  print("y_train : ", data['train'][subtask+'_y_train'].shape)

  print("-" * 10)
  print("x_val:   ", data['validation'][subtask+'_x_val'].shape)
  print("y_val :   ", data['validation'][subtask+'_y_val'].shape)
  print("#" * 10)

Training Set ---- subtask_a
----------
x_train:  (11254,)
y_train :  (11254,)
----------
x_val:    (1986,)
y_val :    (1986,)
##########
Training Set ---- subtask_b
----------
x_train:  (3740,)
y_train :  (3740,)
----------
x_val:    (660,)
y_val :    (660,)
##########
Training Set ---- subtask_c
----------
x_train:  (3294,)
y_train :  (3294,)
----------
x_val:    (582,)
y_val :    (582,)
##########


#Upsampling (prepare balanced data)

In [24]:
def balanceData(subtask):
    ''' Augments the Data given in input in order to balance the dataset'''
    class_size = {}
    for label in classes_dict[subtask]:
        class_size[label] = len(data['train'][subtask+'_x_train'][data['train'][subtask+'_y_train']==classes_dict[subtask][label]])
        print(classes_dict[subtask][label])
    largest_class = max(class_size, key=class_size.get)
    print(largest_class)
    print('---- Augmenting the Data : ')
    print('Before Augmentation : ',class_size)

    for label in classes_dict[subtask]:  
        if label != largest_class:
            id_list = np.where(data['train'][subtask+'_y_train']==classes_dict[subtask][label])[0]
            nb_augmentation = class_size[largest_class] - class_size[label]
            id_augmentation = np.random.choice(id_list, (nb_augmentation,))
            data['train'][subtask+'_x_train']=np.append(data['train'][subtask+'_x_train'],data['train'][subtask+'_x_train'][id_augmentation],axis=0)
            data['train'][subtask+'_y_train']=np.append(data['train'][subtask+'_y_train'],data['train'][subtask+'_y_train'][id_augmentation],axis=0)
    # Check if it went well
    for label in classes_dict[subtask]:
        class_size[label] = len(data['train'][subtask+'_x_train'][data['train'][subtask+'_y_train']==classes_dict[subtask][label]])

    print('After Augmentation : ',class_size)
    return data
for subtask in task_list:
  balanceData(subtask)
#   balanceData(data['train'][subtask],subtask)
  class_size = {}
  for label in classes_dict[subtask]:
    class_size[label] = len(data['train'][subtask+'_y_train'][data['train'][subtask+'_y_train']==classes_dict[subtask][label]])
#   print('**After Augmentation : ',class_size)

0
1
NOT
---- Augmenting the Data : 
Before Augmentation :  {'NOT': 7492, 'OFF': 3762}
After Augmentation :  {'NOT': 7492, 'OFF': 7492}
0
1
TIN
---- Augmenting the Data : 
Before Augmentation :  {'UNT': 452, 'TIN': 3288}
After Augmentation :  {'UNT': 3288, 'TIN': 3288}
0
1
2
IND
---- Augmenting the Data : 
Before Augmentation :  {'IND': 2033, 'OTH': 339, 'GRP': 922}
After Augmentation :  {'IND': 2033, 'OTH': 2033, 'GRP': 2033}


In [0]:
import keras
data['train']['subtask_c'+'_y_train'] = keras.utils.to_categorical(data['train']['subtask_c'+'_y_train'])
data['validation']['subtask_c'+'_y_val'] = keras.utils.to_categorical(data['validation']['subtask_c'+'_y_val'])

In [0]:
from keras.optimizers import Adam
import pickle
from keras.models import *
from keras.layers import *
from keras.callbacks import ModelCheckpoint

#Customized Callback Function

In [0]:
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,accuracy_score
class Metrics(Callback):
    def __init__(self, filepath,validation_data=(),IsBinaryClass=True):
        self.file_path = filepath
        self.X_val, self.y_val = validation_data
        if not IsBinaryClass:
          self.y_val = np.argmax(self.y_val,axis=1)
        self.IsBinaryClass_ = IsBinaryClass
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.best_val_f1 = 0
        self.val_recalls = []
        self.val_precisions = []

    def on_epoch_end(self, epoch, logs={}):
        if self.IsBinaryClass_:
          val_predict = (np.asarray(self.model.predict(self.X_val))).round().astype('int').reshape(-1)
        else:
          val_predict = np.argmax(self.model.predict(self.X_val),axis=1)
        val_targ = self.y_val
        _val_f1 = f1_score(val_targ, val_predict,average='macro')
#         _val_recall = recall_score(val_targ, val_predict)
#         _val_precision = precision_score(val_targ, val_predict)
        self.val_f1s.append(_val_f1)
#         self.val_recalls.append(_val_recall)
#         self.val_precisions.append(_val_precision)
#         print(' — val_f1: %f — val_precision: %f — val_recall %f' %(_val_f1, _val_precision, _val_recall))
        
        print(' — val_f1: %f' %(_val_f1))
        print("max f1; ",max(self.val_f1s))
        if _val_f1 > self.best_val_f1:
            self.model.save(self.file_path, overwrite=True)
            self.best_val_f1 = _val_f1
            print("best f1: {}".format(self.best_val_f1))
        else:
            print("val f1: {}, but not the best f1".format(_val_f1))
        return

#LSTM model

In [0]:
def LSTM_model(IsBinaryClass=True):
  input_data = Input(shape=(83, 768,),dtype='float32')
  merged = LSTM(100,input_dim=(83, 768,))(input_data)

  merged=BatchNormalization()(merged)
  if IsBinaryClass:
    predictions = Dense(1, activation='sigmoid', kernel_initializer='random_normal')(merged)
  else:
    predictions = Dense(3, activation='softmax', kernel_initializer='random_normal')(merged)

  model = Model(
      inputs=input_data,
      outputs=predictions)
  return model

#CNN model

In [0]:
def CNN_binary_class(IsBinaryClass=True):
  input_data = Input(shape=(83, 768,),dtype='float32')
  conv1_x = Conv1D(100, 2,input_dim=(83, 768,), activation='relu')(input_data)
  x_1 =GlobalMaxPooling1D()(conv1_x)
  conv2_x = Conv1D(100, 3,input_dim=(83, 768,), activation='relu')(input_data)
  x_2 = GlobalMaxPooling1D()(conv2_x)
  conv3_x = Conv1D(100, 5,input_dim=(83, 768,), activation='relu')(input_data)
  x_3 = GlobalMaxPooling1D()(conv3_x)

  merged = concatenate([x_1, x_2,x_3],axis=-1)
  merged = Dropout(0.3)(merged)
  merged=BatchNormalization()(merged)
  merged = Dense(units=128,kernel_initializer='random_normal')(merged)
  merged=PReLU()(merged)
#   merged = Dropout(0.2)(merged)
  merged=BatchNormalization()(merged)
  if IsBinaryClass:
    predictions = Dense(1, activation='sigmoid', kernel_initializer='random_normal')(merged)
  else:
    predictions = Dense(3, activation='softmax', kernel_initializer='random_normal')(merged)

  model = Model(
      inputs=input_data,
      outputs=predictions)
  return model

#Set TARGET_TASK = 'subtask_a' , if you want to train task A

In [47]:
TARGET_TASK = 'subtask_c'  # Set TARGET_TASK = 'subtask_a' , if you want to train task
Is_Binary_Class = True
if TARGET_TASK == 'subtask_c':
  Is_Binary_Class = False
MODEL_SAVE_PATH='./output/CNN_Train_'+TARGET_TASK+'_4.hdf5' 
metrics = Metrics(MODEL_SAVE_PATH,
                  validation_data=(bert_data['train'][data['validation'][TARGET_TASK+'_x_val']],data['validation'][TARGET_TASK+'_y_val']),
                  IsBinaryClass=Is_Binary_Class)

model=LSTM_model(IsBinaryClass=Is_Binary_Class)
# model=CNN_binary_class(IsBinaryClass=Is_Binary_Class) # CNN model
model.summary()
# adam1 = keras.optimizers.Adam(lr=0.0001,decay=0.01) # for CNN model
adam1 = keras.optimizers.Adam(lr=0.001,decay=0.0001) # for LSTM model

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(100, input_shape=(None, (83...)`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 83, 768)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               347600    
_________________________________________________________________
batch_normalization_3 (Batch (None, 100)               400       
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 303       
Total params: 348,303
Trainable params: 348,103
Non-trainable params: 200
_________________________________________________________________


#Training

In [48]:
BATCH_SIZE = 300
NUM_EPOCHS = 5
# class_weight = {0: 1.,
#                 1: 2.}
# class_weight = {0: 7.3,
#                 1: 1.}
# class_weight = {0: 1.,
#                 1: 6.1,
#                 2: 2.24
#                }
class_weight = {0: 1.,
                1: 1.5,
                2: 1.5
               }
model.compile(loss='binary_crossentropy',
          optimizer= adam1,
          metrics=['accuracy'])

history = model.fit(
    x=bert_data['train'][data['train'][TARGET_TASK+'_x_train']],
    y=data['train'][TARGET_TASK+'_y_train'],
    batch_size=BATCH_SIZE,
    epochs=NUM_EPOCHS,
    callbacks=[metrics],
    # 每個 epoch 完後計算驗證資料集
    # 上的 Loss 以及準確度
    validation_data=(bert_data['train'][data['validation'][TARGET_TASK+'_x_val']],data['validation'][TARGET_TASK+'_y_val']),
    # 每個 epoch 隨機調整訓練資料集
    # 裡頭的數據以讓訓練過程更穩定
    shuffle=True
)
#     class_weight=class_weight

Train on 6099 samples, validate on 582 samples
Epoch 1/5
6099/6099 [==============================] - 5s 850us/step - loss: 0.5454 - acc: 0.7213 - val_loss: 0.5140 - val_acc: 0.7474
 — val_f1: 0.505084
max f1;  0.5050839643341831
best f1: 0.5050839643341831
Epoch 2/5
6099/6099 [==============================] - 4s 579us/step - loss: 0.4178 - acc: 0.8176 - val_loss: 0.4931 - val_acc: 0.7686
 — val_f1: 0.516072
max f1;  0.5160716990482094
best f1: 0.5160716990482094
Epoch 3/5
6099/6099 [==============================] - 4s 587us/step - loss: 0.3142 - acc: 0.8759 - val_loss: 0.5877 - val_acc: 0.7279
 — val_f1: 0.476753
max f1;  0.5160716990482094
val f1: 0.4767531615903236, but not the best f1
Epoch 4/5
6099/6099 [==============================] - 4s 592us/step - loss: 0.2190 - acc: 0.9226 - val_loss: 0.5766 - val_acc: 0.7904
 — val_f1: 0.504085
max f1;  0.5160716990482094
val f1: 0.504085085535031, but not the best f1
Epoch 5/5
6099/6099 [==============================] - 4s 584us/step -

In [0]:
# file_his = open('./output/CNNV5_train_hist_'+TARGET_TASK+'lstm_4.pickle', 'wb')
# pickle.dump(history.history, file_his)
# file_his.close()

In [49]:
# history.history.keys()
# history.history['f-score']=metrics.val_f1s
# # history.history['best-fscore']=metrics.val_f1s
# history.history.keys()

dict_keys(['val_loss', 'val_acc', 'loss', 'acc', 'f-score'])

In [0]:
# del metrics

#Get The Best Model From Disk

In [51]:
from keras.models import load_model
if os.path.exists(MODEL_SAVE_PATH):
      model=load_model(MODEL_SAVE_PATH)
    # 若成功加载前面保存的参数，输出下列信息
      print("checkpoint_loaded")
predictions = model.predict(bert_data['test'][TARGET_TASK])

checkpoint_loaded


#Evaluate on Testing Set

In [52]:
if TARGET_TASK == 'subtask_c'
  predictions = np.argmax(predictions,axis=1)
else:
  predictions=predictions.round().astype('int').reshape(-1)
print(predictions[:5])

array([1, 2, 0, 0, 0])

In [53]:
TEST_A_LABEL_PATH = DATA_SET_DIR+'labels-levela.csv'
TEST_B_LABEL_PATH = DATA_SET_DIR+'labels-levelb.csv'
TEST_C_LABEL_PATH = DATA_SET_DIR+'labels-levelc.csv'
TEST_LABEL_PATH_DICT = {}
TEST_LABEL_PATH_DICT['subtask_a'] = TEST_A_LABEL_PATH
TEST_LABEL_PATH_DICT['subtask_b'] = TEST_B_LABEL_PATH
TEST_LABEL_PATH_DICT['subtask_c'] = TEST_C_LABEL_PATH

true_label = pd.read_csv(TEST_LABEL_PATH_DICT[TARGET_TASK],sep=',',header=None)
true_label.head(3)

,0,1
0,15923,OTH
1,60133,GRP
2,83681,IND


In [54]:
y_test = true_label[1].apply(lambda x: classes_dict[TARGET_TASK][x])
y_test = np.asarray(y_test).astype('int')
y_test[:5]

array([1, 2, 0, 1, 0])

In [55]:
testing_f1 = f1_score(y_test, predictions,average='macro')
testing_acc = accuracy_score(y_test, predictions)
print('testing data f1-score:',testing_f1)
print('testing data accuracy:',testing_acc)
# testing data f1-score: 0.4489635185344832
# testing data accuracy: 0.49765258215962443
# testing data f1-score: 0.5304134439163793
# testing data accuracy: 0.6150234741784038

testing data f1-score: 0.54145361088442
testing data accuracy: 0.6009389671361502


In [59]:
# !zip lstm_bert_output_c4 output5/*

  adding: output5/CNN_Train_subtask_c_4.hdf5 (deflated 8%)
  adding: output5/CNNV5_train_hist_subtask_clstm_4.pickle (deflated 20%)


In [0]:
# !mv lstm_bert_output_c4.zip './gdrive/My Drive/NLP/Final-Project/output/'

In [0]:
# !mv ./output/CNN_Train_subtask_c_4.hdf5 ./output5/

In [0]:
# !mv ./output/CNNV5_train_hist_subtask_clstm_4.pickle ./output5/

In [0]:
# !unzip -q './gdrive/My Drive/NLP/Final-Project/output/cnn_outputt2_2.zip' -d ./output2

In [0]:
# best_m_p = './output2/output/CNNV5_Train_subtask_b.hdf5'
# if os.path.exists(best_m_p):
#       model=load_model(best_m_p)
#     # 若成功加载前面保存的参数，输出下列信息
#       print("checkpoint_loaded")

In [0]:
# model.summary()

In [0]:
# emb_layer_model = Model(inputs=model.input,outputs=model.get_layer('concatenate_20').output)

In [0]:
# emb_output = emb_layer_model.predict(data['test']['subtask_b'+'_x_test'])
# # emb_output = emb_layer_model.predict(data['train']['subtask_b'+'_token_id'])
# print(emb_output.shape)
# print(emb_output[0])

In [0]:
np.save('cnn_subtaskb_xtest_300',emb_output)
# np.save('cnn_subtaskb_ytest',data['train']['subtask_b'+'_y'])
# np.save('cnn_subtaskb_ytest',y_test)

In [0]:
# emb_output = emb_layer_model.predict(data['train']['subtask_b'+'_token_id'])

# print(emb_output.shape)
# print(emb_output[0])

In [322]:
# print(data['train']['subtask_b'+'_token_id'].shape)
# data['test']['subtask_b'+'_x_test'] = text2id(tokenizer, data['test']['subtask_b'].tweet_exp)
# print(data['test']['subtask_b'+'_x_test'].shape)
# TEST_A_LABEL_PATH = DATA_SET_DIR+'labels-levela.csv'
# TEST_B_LABEL_PATH = DATA_SET_DIR+'labels-levelb.csv'
# TEST_C_LABEL_PATH = DATA_SET_DIR+'labels-levelc.csv'
# TEST_LABEL_PATH_DICT = {}
# TEST_LABEL_PATH_DICT['subtask_a'] = TEST_A_LABEL_PATH
# TEST_LABEL_PATH_DICT['subtask_b'] = TEST_B_LABEL_PATH
# TEST_LABEL_PATH_DICT['subtask_c'] = TEST_C_LABEL_PATH
# temp_task = 'subtask_b'
# true_label = pd.read_csv(TEST_LABEL_PATH_DICT[temp_task],sep=',',header=None)
# y_test = true_label[1].apply(lambda x: classes_dict[temp_task][x])
# y_test = np.asarray(y_test).astype('int')
# print(y_test.shape)

(4400, 66)
all sequence length are  66
(240, 66)
(240,)


In [316]:
# data['train']['subtask_b'+'_y']

array([0., 0., 0., ..., 1., 1., 1.], dtype=float32)

In [22]:
# !zip cnn_bert_output1 output/*

  adding: output/CNN_Train_subtask_a.hdf5 (deflated 9%)
  adding: output/CNN_Train_subtask_b_2.hdf5 (deflated 9%)
  adding: output/CNN_Train_subtask_b_3.hdf5 (deflated 8%)
  adding: output/CNN_Train_subtask_b_3_TRUE.hdf5 (deflated 8%)
  adding: output/CNN_Train_subtask_b_4.hdf5 (deflated 9%)
  adding: output/CNN_Train_subtask_b.hdf5 (deflated 9%)
  adding: output/CNN_Train_subtask_c.hdf5 (deflated 9%)
  adding: output/CNNV5_train_hist_subtask_alstm_1.pickle (deflated 40%)
  adding: output/CNNV5_train_hist_subtask_bcnn_3.pickle (deflated 42%)
  adding: output/CNNV5_train_hist_subtask_bcnn_4.pickle (deflated 43%)
  adding: output/CNNV5_train_hist_subtask_bcnn.pickle (deflated 38%)


In [0]:
# !mv cnn_bert_output1.zip './gdrive/My Drive/NLP/Final-Project/output/'

In [0]:
# !mv ./output/CNN_Train_subtask_b_4.hdf5 ./output/CNN_Train_subtask_b_3_TRUE.hdf5

In [57]:
# !ls ./output/

CNN_Train_subtask_a.hdf5    CNN_Train_subtask_c.hdf5
CNN_Train_subtask_b_2.hdf5  CNNV5_train_hist_subtask_alstm_1.pickle
CNN_Train_subtask_b_3.hdf5  CNNV5_train_hist_subtask_bcnn_3.pickle
CNN_Train_subtask_b_4.hdf5  CNNV5_train_hist_subtask_bcnn.pickle
CNN_Train_subtask_b.hdf5
